In [1]:
!pip install pycaret[full]
!pip install jinja2
!pip install tqdm
!pip install lightgbm
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import

In [2]:
from pycaret import regression

from xgboost import XGBRegressor

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

### Prepare the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe


In [4]:
train_data = pd.read_csv('../../00_original_data/FIFA_train.csv')

#### Preprocess

In [5]:
#contract_until 변수 int 형으로 변환
# 계약 연도만 추출
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:]
    return int(string)


train_data['contract_until'] = train_data['contract_until'].apply(func)

In [6]:
# 문자형 변수 label 인코딩 진행
train_data['continent'] = train_data['continent'].astype('category').cat.codes
train_data['position'] = train_data['position'].astype('category').cat.codes


In [7]:
minmax_data = train_data.copy()

minmax_features = ['reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']
minmax_transformer = MinMaxScaler() # StandardScaler() # RobustScaler()

minmax_data[minmax_features] = minmax_transformer.fit_transform(minmax_data[minmax_features])


In [8]:
standard_data = train_data.copy()

standard_features = ['reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']
standard_transformer = StandardScaler() # RobustScaler()

standard_data[minmax_features] = standard_transformer.fit_transform(standard_data[standard_features])


In [10]:
robust_data = train_data.copy()

robust_features = ['reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']
robust_transformer = RobustScaler()

robust_data[minmax_features] = robust_transformer.fit_transform(robust_data[robust_features])


### Set up PyCaret Regression Model

In [19]:
minmax_model = regression.setup(data=minmax_data[minmax_data.keys().drop(['id', 'name'])], 
              target='value', 
              session_id=42) # Random seed 

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(8932, 10)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6252, 16)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='value',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

In [20]:
xgb = regression.create_model('xgboost')
xgb_tuned = regression.tune_model(xgb, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,289254.3750,5.910647e+11,7.688073e+05,0.9810,0.4893,0.1899
1,274127.2188,5.839918e+11,7.641936e+05,0.9798,0.3526,0.2257
2,268147.9688,5.977824e+11,7.731639e+05,0.9737,0.4165,0.2799
3,278242.3438,4.568607e+11,6.759147e+05,0.9828,0.4441,0.2643
4,278790.2500,6.255143e+11,7.908946e+05,0.9840,0.3353,0.2355
5,287694.1250,6.420635e+11,8.012887e+05,0.9764,0.5172,0.3016
6,378581.3125,1.868085e+12,1.366779e+06,0.9596,0.3753,0.2685
7,348529.3438,1.481148e+12,1.217024e+06,0.9698,0.4813,0.2351
8,269096.6875,4.775751e+11,6.910681e+05,0.9804,0.4617,0.2547


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=4,
             missing=nan, monotone_constraints='()', n_estimators=130,
             n_jobs=-1, num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=42, reg_alpha=0.4, ...)
INFO:logs:tune_model() succesfully completed......................................


In [21]:
m_top_12_models = regression.compare_models(sort='RMSE', n_select=12)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.462430e+05,3.687808e+11,5.579822e+05,0.9900,0.1338,0.0725,1.904
gbr,Gradient Boosting Regressor,2.310033e+05,5.728772e+11,7.281284e+05,0.9827,0.3389,0.2051,0.355
lightgbm,Light Gradient Boosting Machine,1.755717e+05,6.667764e+11,7.615755e+05,0.9813,0.1263,0.0791,0.165
xgboost,Extreme Gradient Boosting,1.739580e+05,8.053691e+11,7.937485e+05,0.9766,0.1203,0.0651,0.750
rf,Random Forest Regressor,1.812743e+05,6.974613e+11,7.979245e+05,0.9799,0.0995,0.0576,1.420
et,Extra Trees Regressor,1.880721e+05,7.420539e+11,8.292278e+05,0.9781,0.1115,0.0599,1.382
dt,Decision Tree Regressor,2.336790e+05,1.282707e+12,1.098519e+06,0.9610,0.1331,0.0648,0.033
ada,AdaBoost Regressor,1.787403e+06,4.429824e+12,2.093499e+06,0.8539,1.5853,5.1668,0.278
llar,Lasso Least Angle Regression,1.811173e+06,1.067001e+13,3.234205e+06,0.6658,1.3209,4.6890,0.021
lasso,Lasso Regression,1.811263e+06,1.067002e+13,3.234209e+06,0.6658,1.3210,4.6894,0.026


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7fe6819e5810>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
          

In [22]:
m_blended12 = regression.blend_models(m_top_12_models, 10, optimize='RMSE')
regression.predict_model(m_blended12)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,680711.0088,1.884275e+12,1.372689e+06,0.9395,0.7386,0.9960
1,637474.5238,1.167830e+12,1.080662e+06,0.9597,0.8488,1.3382
2,592588.2132,9.309290e+11,9.648466e+05,0.9591,0.8198,1.3450
3,628348.9184,1.165072e+12,1.079385e+06,0.9560,0.9248,1.2266
4,722537.8099,2.889156e+12,1.699752e+06,0.9263,0.8221,1.2396
5,654760.4204,1.833223e+12,1.353966e+06,0.9328,1.0048,1.8192
6,720309.2616,2.743633e+12,1.656392e+06,0.9407,0.8609,1.5452
7,735964.0660,3.326233e+12,1.823796e+06,0.9321,0.8150,1.2228
8,615065.5340,1.135114e+12,1.065417e+06,0.9534,0.7294,1.2712


INFO:logs:create_model_container: 23
INFO:logs:master_model_container: 23
INFO:logs:display_container: 5
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe68bb4b0d0>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,715478.382,2.967802e+12,1.722731e+06,0.9232,0.8722,1.6632


,age,continent,position,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,prefer_foot_left,value,Label
0,29.0,4,2,0.00,0.425532,0.413043,0.25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,7.715462e+05
1,24.0,4,2,0.00,0.425532,0.478261,0.50,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,975000.0,1.974831e+06
2,37.0,2,1,0.00,0.531915,0.521739,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,210000.0,6.775707e+05
3,25.0,2,0,0.00,0.425532,0.478261,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,1.171081e+06
4,22.0,0,2,0.00,0.531915,0.673913,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4400000.0,4.575978e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,27.0,2,2,0.00,0.468085,0.456522,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.464014e+06
2676,29.0,2,3,0.25,0.531915,0.521739,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,5.012166e+06
2677,24.0,4,3,0.00,0.382979,0.434783,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,675000.0,1.069470e+06
2678,20.0,2,3,0.00,0.276596,0.478261,0.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,6.833049e+05


In [23]:
m_blended5 = regression.blend_models(m_top_12_models[:5], 10, optimize='RMSE')
regression.predict_model(m_blended5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,153210.9924,1.937132e+11,4.401286e+05,0.9938,0.0961,0.0613
1,156179.4448,6.324574e+11,7.952719e+05,0.9782,0.1500,0.0698
2,133018.6277,1.844152e+11,4.294359e+05,0.9919,0.1466,0.0763
3,127603.0064,2.113053e+11,4.596796e+05,0.9920,0.1011,0.0656
4,151212.6683,4.949723e+11,7.035427e+05,0.9874,0.1203,0.0649
5,135718.0284,2.479481e+11,4.979439e+05,0.9909,0.1735,0.0990
6,207990.0779,7.567120e+11,8.698919e+05,0.9836,0.1749,0.0724
7,226401.7735,1.688270e+12,1.299335e+06,0.9655,0.1380,0.0747
8,155747.7014,4.086809e+11,6.392815e+05,0.9832,0.1257,0.0750


INFO:logs:create_model_container: 24
INFO:logs:master_model_container: 24
INFO:logs:display_container: 7
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe68ba51250>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,160301.6673,9.642011e+11,981937.4025,0.975,0.1889,0.0818


,age,continent,position,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,prefer_foot_left,value,Label
0,29.0,4,2,0.00,0.425532,0.413043,0.25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,6.425380e+05
1,24.0,4,2,0.00,0.425532,0.478261,0.50,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,975000.0,9.403151e+05
2,37.0,2,1,0.00,0.531915,0.521739,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,210000.0,3.952883e+05
3,25.0,2,0,0.00,0.425532,0.478261,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,7.905367e+05
4,22.0,0,2,0.00,0.531915,0.673913,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4400000.0,4.178440e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,27.0,2,2,0.00,0.468085,0.456522,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.086788e+06
2676,29.0,2,3,0.25,0.531915,0.521739,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,3.050323e+06
2677,24.0,4,3,0.00,0.382979,0.434783,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,675000.0,6.571835e+05
2678,20.0,2,3,0.00,0.276596,0.478261,0.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,3.606326e+05


In [24]:
standard_model = regression.setup(data=standard_data[standard_data.keys().drop(['id', 'name'])], 
              target='value', 
              session_id=42) # Random seed 

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(8932, 10)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6252, 16)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='value',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

In [25]:
xgb = regression.create_model('xgboost')
xgb_tuned = regression.tune_model(xgb, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,289898.2188,5.960640e+11,7.720518e+05,0.9809,0.4012,0.1886
1,273162.4375,5.602198e+11,7.484783e+05,0.9806,0.4034,0.2216
2,266949.5625,5.949692e+11,7.713425e+05,0.9738,0.4196,0.2811
3,276583.0000,4.511880e+11,6.717053e+05,0.9830,0.4935,0.2642
4,282464.9062,6.522670e+11,8.076304e+05,0.9834,0.3437,0.2340
5,281191.5312,5.811820e+11,7.623529e+05,0.9787,0.5105,0.3075
6,377378.4062,1.859750e+12,1.363727e+06,0.9598,0.3533,0.2696
7,354710.0625,1.559900e+12,1.248960e+06,0.9682,0.3617,0.2327
8,276292.7500,5.024543e+11,7.088401e+05,0.9794,0.4686,0.2553


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=4,
             missing=nan, monotone_constraints='()', n_estimators=130,
             n_jobs=-1, num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=42, reg_alpha=0.4, ...)
INFO:logs:tune_model() succesfully completed......................................


In [26]:
s_top_12_models = regression.compare_models(sort='RMSE', n_select=12)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.462430e+05,3.687808e+11,5.579822e+05,0.9900,0.1338,0.0725,1.933
gbr,Gradient Boosting Regressor,2.306298e+05,5.655259e+11,7.218023e+05,0.9830,0.3389,0.2051,0.362
lightgbm,Light Gradient Boosting Machine,1.755717e+05,6.667764e+11,7.615755e+05,0.9813,0.1263,0.0791,0.162
xgboost,Extreme Gradient Boosting,1.737831e+05,8.034175e+11,7.915420e+05,0.9766,0.1203,0.0651,0.764
rf,Random Forest Regressor,1.809391e+05,6.903411e+11,7.929071e+05,0.9801,0.0997,0.0577,1.427
et,Extra Trees Regressor,1.880721e+05,7.420539e+11,8.292278e+05,0.9781,0.1115,0.0599,1.452
dt,Decision Tree Regressor,2.334879e+05,1.282462e+12,1.098416e+06,0.9610,0.1335,0.0648,0.039
ada,AdaBoost Regressor,1.787115e+06,4.428053e+12,2.093055e+06,0.8540,1.5853,5.1669,0.297
knn,K Neighbors Regressor,8.136391e+05,5.009783e+12,2.200681e+06,0.8434,0.3851,0.3411,0.076
ridge,Ridge Regression,1.813213e+06,1.066638e+13,3.233814e+06,0.6659,1.3251,4.6953,0.019


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7fe6831e4410>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
          

In [27]:
s_blended12 = regression.blend_models(s_top_12_models, 10, optimize='RMSE')
regression.predict_model(s_blended12)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,580028.9021,1.575479e+12,1.255181e+06,0.9494,0.6650,0.7324
1,541654.2871,9.487034e+11,9.740141e+05,0.9672,0.6714,0.9746
2,496191.0284,7.332674e+11,8.563103e+05,0.9678,0.8230,0.9814
3,527739.1304,8.505707e+11,9.222639e+05,0.9679,0.7706,0.9073
4,615532.4824,2.448096e+12,1.564639e+06,0.9375,0.7050,0.8982
5,542046.7447,1.419555e+12,1.191451e+06,0.9479,0.8133,1.3215
6,617853.4138,2.288935e+12,1.512923e+06,0.9505,0.7438,1.1224
7,638840.5781,2.940672e+12,1.714839e+06,0.9400,0.6533,0.9049
8,528034.4789,8.976003e+11,9.474177e+05,0.9631,0.6377,0.9339


INFO:logs:create_model_container: 23
INFO:logs:master_model_container: 23
INFO:logs:display_container: 5
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe683233f90>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,604761.2007,2.472630e+12,1.572460e+06,0.936,0.7826,1.2162


,age,continent,position,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,prefer_foot_left,value,Label
0,29.0,4,2,-0.308843,-0.013295,-0.834562,-0.517654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,7.840972e+05
1,24.0,4,2,-0.308843,-0.013295,-0.333544,0.770999,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,975000.0,1.760039e+06
2,37.0,2,1,-0.308843,0.716150,0.000467,-1.806307,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,210000.0,6.373829e+05
3,25.0,2,0,-0.308843,-0.013295,-0.333544,-0.517654,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,1.156322e+06
4,22.0,0,2,-0.308843,0.716150,1.169509,0.770999,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4400000.0,4.571435e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,27.0,2,2,-0.308843,0.278483,-0.500550,0.770999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.428201e+06
2676,29.0,2,3,2.050938,0.716150,0.000467,0.770999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,4.653511e+06
2677,24.0,4,3,-0.308843,-0.305073,-0.667556,-0.517654,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,675000.0,1.008513e+06
2678,20.0,2,3,-0.308843,-1.034518,-0.333544,0.770999,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,6.499067e+05


In [28]:
s_blended5 = regression.blend_models(s_top_12_models[:5], 10, optimize='RMSE')
regression.predict_model(s_blended5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,153383.7880,1.955615e+11,4.422234e+05,0.9937,0.0962,0.0613
1,156275.4315,6.331567e+11,7.957114e+05,0.9781,0.1499,0.0697
2,132967.7587,1.842324e+11,4.292230e+05,0.9919,0.1466,0.0763
3,127743.6915,2.111241e+11,4.594824e+05,0.9920,0.1011,0.0656
4,150830.8674,4.864770e+11,6.974790e+05,0.9876,0.1202,0.0648
5,135283.7527,2.416502e+11,4.915793e+05,0.9911,0.1735,0.0990
6,207786.1457,7.556167e+11,8.692621e+05,0.9837,0.1744,0.0724
7,226266.7775,1.687730e+12,1.299127e+06,0.9656,0.1380,0.0747
8,156104.3770,4.156943e+11,6.447436e+05,0.9829,0.1256,0.0750


INFO:logs:create_model_container: 24
INFO:logs:master_model_container: 24
INFO:logs:display_container: 7
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe680571650>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,160322.7127,9.643060e+11,981990.8354,0.975,0.1892,0.0817


,age,continent,position,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,prefer_foot_left,value,Label
0,29.0,4,2,-0.308843,-0.013295,-0.834562,-0.517654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,6.425380e+05
1,24.0,4,2,-0.308843,-0.013295,-0.333544,0.770999,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,975000.0,9.403151e+05
2,37.0,2,1,-0.308843,0.716150,0.000467,-1.806307,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,210000.0,3.952883e+05
3,25.0,2,0,-0.308843,-0.013295,-0.333544,-0.517654,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,7.905367e+05
4,22.0,0,2,-0.308843,0.716150,1.169509,0.770999,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4400000.0,4.176040e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,27.0,2,2,-0.308843,0.278483,-0.500550,0.770999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.086788e+06
2676,29.0,2,3,2.050938,0.716150,0.000467,0.770999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,3.050323e+06
2677,24.0,4,3,-0.308843,-0.305073,-0.667556,-0.517654,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,675000.0,6.571835e+05
2678,20.0,2,3,-0.308843,-1.034518,-0.333544,0.770999,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,3.606326e+05


In [29]:
robust_model = regression.setup(data=robust_data[robust_data.keys().drop(['id', 'name'])], 
              target='value', 
              session_id=42) # Random seed 

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(8932, 10)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6252, 16)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='value',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

In [30]:
xgb = regression.create_model('xgboost')
xgb_tuned = regression.tune_model(xgb, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,285235.3750,5.810289e+11,7.622526e+05,0.9814,0.4193,0.1902
1,274901.2812,5.897108e+11,7.679263e+05,0.9796,0.4131,0.2270
2,268297.2500,6.015953e+11,7.756258e+05,0.9736,0.4198,0.2806
3,276052.8438,4.594014e+11,6.777916e+05,0.9827,0.4645,0.2625
4,278271.6250,6.150292e+11,7.842380e+05,0.9843,0.3389,0.2356
5,286062.6250,6.283468e+11,7.926832e+05,0.9770,0.5935,0.2999
6,376431.5000,1.827488e+12,1.351846e+06,0.9605,0.3715,0.2688
7,348655.0625,1.519168e+12,1.232546e+06,0.9690,0.3305,0.2358
8,275765.9375,5.008231e+11,7.076886e+05,0.9794,0.4408,0.2572


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=4,
             missing=nan, monotone_constraints='()', n_estimators=130,
             n_jobs=-1, num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=42, reg_alpha=0.4, ...)
INFO:logs:tune_model() succesfully completed......................................


In [31]:
r_top_12_models = regression.compare_models(sort='RMSE', n_select=12)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.462430e+05,3.687808e+11,5.579822e+05,0.9900,0.1338,0.0725,1.885
gbr,Gradient Boosting Regressor,2.310033e+05,5.728772e+11,7.281284e+05,0.9827,0.3389,0.2051,0.373
lightgbm,Light Gradient Boosting Machine,1.755717e+05,6.667764e+11,7.615755e+05,0.9813,0.1263,0.0791,0.088
xgboost,Extreme Gradient Boosting,1.737338e+05,8.033330e+11,7.914612e+05,0.9766,0.1221,0.0651,0.757
rf,Random Forest Regressor,1.812001e+05,6.951852e+11,7.968538e+05,0.9800,0.0996,0.0576,1.429
et,Extra Trees Regressor,1.880721e+05,7.420539e+11,8.292278e+05,0.9781,0.1115,0.0599,1.394
dt,Decision Tree Regressor,2.336806e+05,1.282708e+12,1.098519e+06,0.9610,0.1331,0.0648,0.036
ada,AdaBoost Regressor,1.799840e+06,4.456978e+12,2.101298e+06,0.8541,1.5929,5.2299,0.279
knn,K Neighbors Regressor,9.289312e+05,7.224718e+12,2.640013e+06,0.7787,0.4592,0.4179,0.073
ridge,Ridge Regression,1.812939e+06,1.066642e+13,3.233789e+06,0.6659,1.3337,4.6955,0.016


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7fe680503990>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
          

In [32]:
r_blended12 = regression.blend_models(r_top_12_models, 10, optimize='RMSE')
regression.predict_model(r_blended12)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,576688.8616,1.575280e+12,1.255102e+06,0.9494,0.6823,0.7316
1,547153.3844,9.623490e+11,9.809939e+05,0.9668,0.6804,0.9738
2,493479.2801,7.231973e+11,8.504101e+05,0.9682,0.7917,0.9810
3,535253.0815,9.908355e+11,9.954072e+05,0.9626,0.7305,0.9072
4,623777.5951,2.644823e+12,1.626291e+06,0.9325,0.7063,0.8979
5,546442.9065,1.530440e+12,1.237110e+06,0.9439,0.8396,1.3218
6,632290.4398,2.463457e+12,1.569540e+06,0.9468,0.7245,1.1004
7,638094.3641,3.089517e+12,1.757702e+06,0.9369,0.6502,0.9049
8,523515.6194,9.154109e+11,9.567711e+05,0.9624,0.6456,0.9345


INFO:logs:create_model_container: 23
INFO:logs:master_model_container: 23
INFO:logs:display_container: 5
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe6804666d0>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,607837.0726,2.549411e+12,1.596687e+06,0.934,0.7802,1.2146


,age,continent,position,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,prefer_foot_left,value,Label
0,29.0,4,2,0.0,0.000000,-0.625,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,8.115846e+05
1,24.0,4,2,0.0,0.000000,-0.250,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,975000.0,1.767849e+06
2,37.0,2,1,0.0,0.555556,0.000,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,210000.0,6.436219e+05
3,25.0,2,0,0.0,0.000000,-0.250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,1.156269e+06
4,22.0,0,2,0.0,0.555556,0.875,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4400000.0,4.519250e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,27.0,2,2,0.0,0.222222,-0.375,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.428173e+06
2676,29.0,2,3,1.0,0.555556,0.000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,4.465986e+06
2677,24.0,4,3,0.0,-0.222222,-0.500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,675000.0,1.004995e+06
2678,20.0,2,3,0.0,-0.777778,-0.250,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,6.364445e+05


In [33]:
r_blended5 = regression.blend_models(r_top_12_models[:5], 10, optimize='RMSE')
regression.predict_model(r_blended5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,153228.9317,1.940057e+11,4.404608e+05,0.9938,0.0962,0.0613
1,156050.4352,6.295974e+11,7.934718e+05,0.9783,0.1499,0.0697
2,132899.3923,1.841274e+11,4.291007e+05,0.9919,0.1466,0.0762
3,127648.7184,2.116884e+11,4.600961e+05,0.9920,0.1011,0.0656
4,151162.3999,4.943543e+11,7.031033e+05,0.9874,0.1202,0.0648
5,135303.2832,2.417626e+11,4.916936e+05,0.9911,0.1733,0.0989
6,207990.0657,7.560794e+11,8.695282e+05,0.9837,0.1749,0.0724
7,226336.9415,1.688133e+12,1.299282e+06,0.9655,0.1380,0.0747
8,155721.7476,4.084736e+11,6.391194e+05,0.9832,0.1257,0.0750


INFO:logs:create_model_container: 24
INFO:logs:master_model_container: 24
INFO:logs:display_container: 7
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe68165d8d0>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,160405.3253,9.647226e+11,982202.918,0.975,0.1889,0.0817


,age,continent,position,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,prefer_foot_left,value,Label
0,29.0,4,2,0.0,0.000000,-0.625,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,6.425380e+05
1,24.0,4,2,0.0,0.000000,-0.250,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,975000.0,9.403151e+05
2,37.0,2,1,0.0,0.555556,0.000,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,210000.0,3.959183e+05
3,25.0,2,0,0.0,0.000000,-0.250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,7.905367e+05
4,22.0,0,2,0.0,0.555556,0.875,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4400000.0,4.178440e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,27.0,2,2,0.0,0.222222,-0.375,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.086788e+06
2676,29.0,2,3,1.0,0.555556,0.000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,3.050323e+06
2677,24.0,4,3,0.0,-0.222222,-0.500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,675000.0,6.571835e+05
2678,20.0,2,3,0.0,-0.777778,-0.250,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,3.606326e+05


## + Additional Model

### Catboost

pycaret 진행 결과 가장 에러가 적었기에 따로 생성

In [34]:
ctb = regression.create_model('catboost')
ctb_tuned = regression.tune_model(ctb, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,272090.0328,2.893483e+11,537911.0547,0.9907,0.5057,0.2420
1,284717.2423,7.640525e+11,874100.9597,0.9736,0.4657,0.2430
2,240039.3830,3.613126e+11,601092.8700,0.9841,0.4746,0.2308
3,248791.2568,3.981527e+11,630993.4418,0.9850,0.4244,0.2322
4,245986.5324,3.276884e+11,572440.7282,0.9916,0.3598,0.2245
5,244559.4890,2.625751e+11,512420.8091,0.9904,0.3847,0.2868
6,307555.3766,8.061479e+11,897857.4074,0.9826,0.4166,0.2939
7,317330.1057,7.412521e+11,860959.9929,0.9849,0.4075,0.2523
8,230689.9045,3.256520e+11,570659.2480,0.9866,0.3875,0.2052


INFO:logs:create_model_container: 26
INFO:logs:master_model_container: 26
INFO:logs:display_container: 10
INFO:logs:<catboost.core.CatBoostRegressor object at 0x7fe6805685d0>
INFO:logs:tune_model() succesfully completed......................................


In [42]:
# 결과 모델 점수

ctb_tuned.get_best_score()

{'learn': {'RMSE': 436812.4962011265}}

In [44]:
for i in range(len(ctb_tuned.feature_names_)):
  print(ctb_tuned.feature_names_[i], ctb_tuned.feature_importances_[i])

age 4.412556313658922
continent 0.012758788472104815
position 1.067898840965457
reputation 0.01352177170898384
stat_overall 74.58936047363402
stat_potential 18.85865762024047
stat_skill_moves 1.0123572889371903
contract_until_2018 0.0
contract_until_2019 0.01820249947320363
contract_until_2020 0.0
contract_until_2021 0.0
contract_until_2022 0.0
contract_until_2023 0.0
contract_until_2024 0.000788341234940824
contract_until_2025 0.0027864769681562995
prefer_foot_left 0.011111584706553022
